## Using Llama 3.1 on AWS to build an Agent

In [1]:
#install required libraries if needed
#!pip3 install openmeteo-requests retry-requests geopy

In [2]:
#load our secrets for AWS
import json
with open('config.json') as f:
    secrets = json.load(f)

In [3]:
#initialize our model and imports
import boto3
from botocore.exceptions import ClientError
import openmeteo_requests
import json

from retry_requests import retry
from requests import Session

client = boto3.client(service_name='bedrock-runtime', region_name="us-west-2",aws_access_key_id=secrets['aws_access_key_id'], aws_secret_access_key=secrets['aws_secret_access_key']) 
#model_id = "meta.llama3-1-70b-instruct-v1:0"
model_id = "meta.llama3-1-405b-instruct-v1:0"
#model_id = "anthropic.claude-3-opus-20240229-v1:0"


### A toy tool calling example to demonstrate the converse API

In [4]:
radio_tools = {
    "tools": [
        {
            "toolSpec": {
                "name": "top_song",
                "description": "Get the most popular song played on a radio station.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "sign": {
                                "type": "string",
                                "description": "The call sign for the radio station for which you want the most popular song. Example calls signs are WZPZ and WKRP."
                            }
                        },
                        "required": [
                            "sign"
                        ]
                    }
                }
            }
        }
    ]
}

conversation = [
    {
        "role": "user",
        "content": [
            {
                "text": "What is the most popular song on WZPZ?"
            }
        ]
    }
]


# Send the message to the model, using a basic inference configuration.
response = client.converse(
    toolConfig = radio_tools,
    modelId=model_id,
    messages=conversation,
    inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
)


In [5]:
response

{'ResponseMetadata': {'RequestId': '42c11e67-8412-4615-8817-9654e45d9f8d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 15 Aug 2024 17:15:33 GMT',
   'content-type': 'application/json',
   'content-length': '273',
   'connection': 'keep-alive',
   'x-amzn-requestid': '42c11e67-8412-4615-8817-9654e45d9f8d'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'toolUse': {'toolUseId': 'tooluse_Y1YbwUU5TlW68Gfsf-7Z1g',
      'name': 'top_song',
      'input': {'sign': 'WZPZ'}}}]}},
 'stopReason': 'tool_use',
 'usage': {'inputTokens': 103, 'outputTokens': 28, 'totalTokens': 131},
 'metrics': {'latencyMs': 2710}}

### Function Inspection
Now we need to define a function that can interrogate Python functions and provide a toolspec according to the converse API definition

In [6]:
# explicit function
def fun(a: int, b: int):
    return a**b

In [7]:
import inspect

def function_inspection(func) -> dict:
    field_types = {
        str: "string",
        int: "integer",
        float: "float",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Error retrieving function signature for {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = field_types.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type {param.annotation} for {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]
    if "kwargs" in required: required.remove("kwargs")

    return {
        "toolSpec": {
            "name": func.__name__,
            "description": func.__doc__ or "",
            "inputSchema": {
                "json": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
            },
        },
    }


In [8]:
function_inspection(fun)

{'toolSpec': {'name': 'fun',
  'description': '',
  'inputSchema': {'json': {'type': 'object',
    'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}},
    'required': ['a', 'b']}}}}

### Getting the current weather by lat/long
Using open-meteo we can check the current weather for a given location

In [9]:
import openmeteo_requests

from retry_requests import retry
from requests import Session

# Setup the Open-Meteo API client with cache and retry on error
retry_session = retry(Session(), retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"current": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()

print(f"Current time {current.Time()}")
print(f"Current temperature_2m {current_temperature_2m}")

Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Current time 1723742100
Current temperature_2m 26.600000381469727


In [10]:
def current_weather(latitude:float, longitude:float, place:str="") -> str:
    """Gets the current temperature for a location given a specific location's latitude and longitude, pass the full floating point number values in"""
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": "temperature_2m"
    }
    responses = openmeteo.weather_api(url, params=params)

    response = responses[0]
    current = response.Current()
    current_temperature_2m = current.Variables(0).Value()

    return "The current temperature at %s is %f degrees." %(place, current_temperature_2m)

In [11]:
current_weather(47.4864, 122.1943)

'The current temperature at  is 17.000000 degrees.'

In [12]:
function_inspection(current_weather)

{'toolSpec': {'name': 'current_weather',
  'description': "Gets the current temperature for a location given a specific location's latitude and longitude, pass the full floating point number values in",
  'inputSchema': {'json': {'type': 'object',
    'properties': {'latitude': {'type': 'float'},
     'longitude': {'type': 'float'},
     'place': {'type': 'string'}},
    'required': ['latitude', 'longitude']}}}}

### Building a basic agent
We can build a single tooled agent that can reply with the current temperature for a given location, or more specifically, identify the function and inputs that need to be called

In [13]:
policy = """You are a weather assistant agent - users ask for a variety of information including the temperature, humidity, and preciptation that is weather related.  
You have access to current, past, and future weather using Functions, 
do not answer from memory for weather related information, instead collect the required data and call the Function appropriately"""

In [14]:
from pydantic import BaseModel, Field
from typing import List, Callable, Dict
class Agent(BaseModel):
    name: str = 'Agent'
    llm_model_id: str = "meta.llama3-1-405b-instruct-v1:0"
    policy: str = "You are an AI assistant."
    tools: Dict = {}
    tools_json: Dict = {}

    def build_tool_json(self):
        tools_json = {"tools": []}
        for k,v in self.tools.items():
            tools_json['tools'].append(function_inspection(v))
        self.tools_json = tools_json


In [15]:
weather_agent = Agent(name="Weather Agent", policy=policy,tools={"current_weather": current_weather})

In [16]:
weather_agent.build_tool_json()

In [17]:
weather_agent.tools_json

{'tools': [{'toolSpec': {'name': 'current_weather',
    'description': "Gets the current temperature for a location given a specific location's latitude and longitude, pass the full floating point number values in",
    'inputSchema': {'json': {'type': 'object',
      'properties': {'latitude': {'type': 'float'},
       'longitude': {'type': 'float'},
       'place': {'type': 'string'}},
      'required': ['latitude', 'longitude']}}}}]}

In [18]:
# Start a conversation with the user message.
user_message = "What's the temperature for lat 47.4864 and long 122.1943?"
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]


# Send the message to the model, using a basic inference configuration.
response = client.converse(
    system=[{"text": weather_agent.policy}],
    modelId=weather_agent.llm_model_id,
    messages=conversation,
    toolConfig = weather_agent.tools_json,
    inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},)


In [19]:
response

{'ResponseMetadata': {'RequestId': '351e6d97-91b5-4d3b-835b-c9d169392583',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 15 Aug 2024 17:15:39 GMT',
   'content-type': 'application/json',
   'content-length': '300',
   'connection': 'keep-alive',
   'x-amzn-requestid': '351e6d97-91b5-4d3b-835b-c9d169392583'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'toolUse': {'toolUseId': 'tooluse_VUSXvM37RKC1UFJHsh3eWw',
      'name': 'current_weather',
      'input': {'latitude': '47', 'longitude': '122'}}}]}},
 'stopReason': 'tool_use',
 'usage': {'inputTokens': 172, 'outputTokens': 31, 'totalTokens': 203},
 'metrics': {'latencyMs': 2948}}

In [20]:
#calling a tool and providing the required inputs as a JSON object, similar to that returned above, demonstrating how we will dynamically call the tools with teh toolUse response
weather_agent.tools["current_weather"](**{'latitude': '47', 'longitude': '122'})

'The current temperature at  is 18.950001 degrees.'

### Location Lookup
Nobody wants to really search for weather by lat long, so we need another tool to find the coordinates for a user passed location

In [21]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")
city ="Fairwood, WA"
loc = geolocator.geocode(city)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

latitude is :- 47.4468672 
longtitude is:- -122.1525349


In [22]:
#turning that into a function we can use
from geopy.geocoders import Nominatim
def geolocation(location: str) -> str:
    """Input user location with as much specificity as is known, if a city and a state are provided, submit in the form of City, State"""
    geolocator = Nominatim(user_agent="location_agent")
    loc = geolocator.geocode(location)
    return "The latitude and longitude for %s is %f, %f" % (location, loc.latitude, loc.longitude)

### Building a multi-tool agent
Now we have 2 tools and can write a prompt and build our agent to be empowered with both of these

In [23]:
policy = """You are a weather assistant agent - users ask for a variety of information including the temperature, humidity, and preciptation that is weather related.  
Use tools where necessary to gather more information if it is not provided by the user or in this prompt."""

weather_agent = Agent(name="Weather Agent", policy=policy,tools={"current_weather": current_weather, "geolocation":geolocation})
weather_agent.build_tool_json()

### Inferencing
We want to invoke the converse API and any time it returns a toolUse response, we want to call that tool with the output, add that to the conversation, and call the converse API again

In [24]:
class Inference(BaseModel):
    conversation: List
    agent: Agent
    system_prompt:str = ""

    def set_prompt(self):
        self.system_prompt = self.agent.policy

    def run_inference(self):
        response = client.converse(
            system=[{"text": self.system_prompt}],
            modelId=self.agent.llm_model_id,
            messages=self.conversation,
            toolConfig = self.agent.tools_json,
            inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},)

        print(json.dumps(response, indent=2))
        
        if "toolUse" in response['output']['message']['content'][0].keys():
            tool_name = response['output']['message']['content'][0]['toolUse']['name']
            tool_inputs = response['output']['message']['content'][0]['toolUse']['input']

            tool_result = weather_agent.tools[tool_name](**tool_inputs)

            print("called tool with result " + str(tool_result) + " Is this enough information to answer the question?")

            #self.system_prompt = self.system_prompt+" "+tool_result
            self.conversation.append({"role": "assistant",'content': [{'text':"Called tool %s with result %s" %(tool_name, tool_result)}]})
            self.conversation.append({"role": "user",'content': [{'text':"Is this enough information to answer the question?"}]})

            return("tool_call")
        
        else:
            response_text = response["output"]["message"]["content"][0]["text"]
            print(response_text)

            self.conversation.append({"role": "assistant",'content': [{'text':response_text}]})

            print(self.conversation)

        return response_text

    def run(self):
        response = self.run_inference()
        while response == "tool_call":
            response = self.run_inference()
        print(response)


In [25]:
policy = """You are a weather assistant agent - users ask for a variety of information including the temperature, humidity, and preciptation that is weather related.  
Use tools where necessary to gather more information if it is not provided by the user or in this prompt."""

weather_agent = Agent(name="Weather Agent", policy=policy,tools={"current_weather": current_weather, "geolocation":geolocation})
weather_agent.build_tool_json()

In [26]:
conversation = [
    {
        "role": "user",
        "content": [{"text": "What's the current temperature in Atlanta Georgia?"}],
    }
    ]

weather_conversation = Inference(conversation=conversation, agent=weather_agent)
weather_conversation.set_prompt()
weather_conversation.run()

{
  "ResponseMetadata": {
    "RequestId": "c4f40af2-2c7e-4ec4-bc14-bfc68ac53766",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 15 Aug 2024 17:15:42 GMT",
      "content-type": "application/json",
      "content-length": "291",
      "connection": "keep-alive",
      "x-amzn-requestid": "c4f40af2-2c7e-4ec4-bc14-bfc68ac53766"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "toolUse": {
            "toolUseId": "tooluse_3yFxa9cJQhGxlZuq6diDAQ",
            "name": "geolocation",
            "input": {
              "location": "Atlanta Georgia"
            }
          }
        }
      ]
    }
  },
  "stopReason": "tool_use",
  "usage": {
    "inputTokens": 212,
    "outputTokens": 26,
    "totalTokens": 238
  },
  "metrics": {
    "latencyMs": 2535
  }
}
called tool with result The latitude and longitude for Atlanta Georgia is 33.748992, -84.390264 Is this enough information to ans